<a href="https://colab.research.google.com/github/Aakash326/ML-models/blob/main/Compny_brocher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install google-generativeai python-dotenv beautifulsoup4 requests

In [2]:
GEMINI_API_KEY='***************'

In [3]:
import os
import json
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import google.generativeai as genai
from IPython.display import Markdown, display

In [4]:
import google.generativeai as genai

# ✅ Paste your Gemini key directly here
api_key = "*************"
genai.configure(api_key=api_key)

In [5]:
if api_key and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please check your .env file!")

MODEL = 'gemini-1.5-flash'
genai.configure(api_key=api_key)

API key looks good so far


In [6]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [7]:
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [8]:
ed=Website("https://www.gitam.edu/hyderabad/campus-life")
print(ed.links)

['/aspiring-students', '/bengaluru/campus-life', 'https://www.alumni.gitam.edu/', '/about/faculty', 'https://vdc.gitam.edu/', '/faqs', 'https://careers.gitam.edu/', 'javascript:;', 'https://login.gitam.edu/', 'https://gparent.gitam.edu/login', 'https://login.gitam.edu/Login.aspx', 'https://gpay.gitam.edu/', '/', 'javascript:;', 'https://login.gitam.edu/', 'https://gparent.gitam.edu/login', 'https://login.gitam.edu/Login.aspx', 'https://gpay.gitam.edu/', '/about-us', 'javascript:;', '/bengaluru/campus-life', '/visakhapatnam/campus-life', '/hyderabad/campus-life', '/gimsr/campus-life', '/about/faculty', '/about/leadership', '/about/accrediation', 'https://iqac.gitam.edu/nirf', '/about/sustainable-development', 'javascript:;', 'https://iqac.gitam.edu/sustainable_climate', 'https://iqac.gitam.edu/GITAM_Sustainable_Procurement', 'https://iqac.gitam.edu/sustainable_investment', 'https://iqac.gitam.edu/net_zero_policy', 'https://iqac.gitam.edu/equality_diversity_policy', 'https://iqac.gitam.e

In [20]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs,placments as this is important .\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [21]:
link_system_prompt

'You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs,placments as this is important .\nYou should respond in JSON as in this example:\n{\n    "links": [\n        {"type": "about page", "url": "https://full.url/goes/here/about"},\n        {"type": "careers page", "url": "https://another.full.url/careers"}\n    ]\n}\n'

In [22]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [23]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.gitam.edu/hyderabad/campus-life - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/aspiring-students
/bengaluru/campus-life
https://www.alumni.gitam.edu/
/about/faculty
https://vdc.gitam.edu/
/faqs
https://careers.gitam.edu/
javascript:;
https://login.gitam.edu/
https://gparent.gitam.edu/login
https://login.gitam.edu/Login.aspx
https://gpay.gitam.edu/
/
javascript:;
https://login.gitam.edu/
https://gparent.gitam.edu/login
https://login.gitam.edu/Login.aspx
https://gpay.gitam.edu/
/about-us
javascript:;
/bengaluru/campus-life
/visakhapatnam/campus-life
/hyderabad/campus-life
/gimsr/campus-life
/about/faculty
/about/leadership
/about/accrediation
https://iqac.gitam.edu/nirf
/about/sustainable-development
javascript:;
https://iqac.gitam.edu/sustainable_clima

In [24]:
def get_links(url):
    website = Website(url)
    model = genai.GenerativeModel(MODEL)
    # Combine system and user prompts into a single user prompt
    combined_prompt = link_system_prompt + "\n" + get_links_user_prompt(website)
    response = model.generate_content([
        {"role": "user", "parts": [combined_prompt]}
    ])
    result = response.text
    return json.loads(result)

In [25]:
import json
import re

def get_links(url):
    website = Website(url)
    model = genai.GenerativeModel(MODEL)

    # Combine system + user prompt as single user prompt (Gemini-style)
    combined_prompt = link_system_prompt + "\n" + get_links_user_prompt(website)
    response = model.generate_content([
        {"role": "user", "parts": [combined_prompt]}
    ])

    # Try to extract JSON block from text
    text = response.text.strip()

    try:
        # Look for the first valid JSON block
        json_text = re.search(r'\{[\s\S]*\}', text).group()
        return json.loads(json_text)
    except Exception as e:
        print("❌ Could not parse JSON from Gemini output:")
        print(text)
        raise e

In [26]:
get_links('https://www.gitam.edu/hyderabad/campus-life')

{'links': [{'type': 'about us', 'url': 'https://www.gitam.edu/about-us'},
  {'type': 'careers page', 'url': 'https://careers.gitam.edu/'},
  {'type': 'alumni page', 'url': 'https://www.alumni.gitam.edu/'}]}

In [27]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [28]:
get_all_details('https://www.gitam.edu/hyderabad/campus-life')

Found links: {'links': [{'type': 'about us page', 'url': 'https://www.gitam.edu/about-us'}, {'type': 'careers page', 'url': 'https://careers.gitam.edu/'}, {'type': 'alumni page', 'url': 'https://www.alumni.gitam.edu/'}]}


"Landing page:\nWebpage Title:\nLife at GITAM Hyderabad | Campus Life\nWebpage Contents:\nInformation For:\nAspiring Students\nCampus Life\nAlumni\nFaculty\nEntrepreneurship\nFAQs\nCareers\nEmployee Login\nParent Login\nStudent Login\nPayments\nEmployee Login\nParent Login\nStudent Login\nPayments\nAbout Us\nAbout GITAM\nCampus Life\nBengaluru\nVisakhapatnam\nHyderabad\nGIMSR\nFaculty\nLeadership\nAccreditation\nApprovals\nNIRF\nSustainable Development\nPolicies\nClimate Action\nSustainable Procurement\nSustainable Investment\nNet Zero Commitment\nEquality, Diversity and Inclusiveness\nCommittee for Differently Abled\nEqual Opportunity Cell\nWomen Empowerment Cell\nFeedback\nITEP (NCTE)\nAcademics\nBengaluru\nEngineering\nHumanities and Social Sciences\nManagement\nScience\nHyderabad\nArchitecture\nEducation\nEngineering\nHumanities and Social Sciences\nManagement\nPharmacy\nPublic Policy\nScience\nVisakhapatnam\nArchitecture\nEngineering\nHumanities and Social Sciences\nLaw\nManagemen

In [45]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors,recruits and the contact information is important that are mentioned in the website. Respond in markdown.\
Include details of company culture, customers and careers/jobs, placements if you have the information ."

In [54]:
system_prompt = """
You are an assistant that analyzes the contents of a company website and creates a brochure.

Instructions:
- The brochure should be in Markdown.
- Write all sections in English **except** the "Academic Excellence" section, which must be written in **Telugu**.
- Translate only the section title and content under "Academic Excellence".
- Make the output structured, readable, and inspiring.

Here is an example brochure structure:

## About Us
{English content here...}

## Academic Excellence
{తెలుగులో మాత్రమే ఈ భాగాన్ని రాయండి. ఉదాహరణకు: మా విద్యా ప్రమాణాలు దేశంలోనే అత్యుత్తమంగా ఉన్నాయి.}

## Careers
{English content again...}
"""

In [55]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000]  # Truncate if too long
    return user_prompt


In [56]:
import google.generativeai as genai
from IPython.display import Markdown, display

def create_brochure(company_name, url):
    model = genai.GenerativeModel("gemini-1.5-flash")  # ✅ use flash model

    # Combine both system and user prompt into one message
    prompt = (
        system_prompt + "\n\n" +
        get_brochure_user_prompt(company_name, url)+ multi_shot
    )

    # Gemini accepts a single list of messages or just one string
    response = model.generate_content(prompt)

    # Show the result
    display(Markdown(response.text))

In [57]:
create_brochure('Gitam', 'https://www.gitam.edu/hyderabad/campus-life')

Found links: {'links': [{'type': 'about us page', 'url': 'https://www.gitam.edu/about-us'}, {'type': 'careers page', 'url': 'https://careers.gitam.edu/'}, {'type': 'alumni page', 'url': 'https://www.alumni.gitam.edu/'}]}


## GITAM Hyderabad: Shaping Future Leaders

**A Glimpse into Excellence**

Established in 2009, GITAM (Deemed to be University) Hyderabad has rapidly become a leading institution of higher education in Telangana.  With seven schools offering diverse programs in Technology, Management, Pharmacy, Architecture, and more, GITAM Hyderabad provides a vibrant and enriching learning environment for students from across the globe.  Our commitment to academic excellence, coupled with state-of-the-art infrastructure and a strong focus on student well-being, sets us apart.  We foster a culture of innovation, collaboration, and global engagement, preparing our graduates for successful and fulfilling careers.


## Academic Excellence

**విద్యా ప్రతిభ:**

గిటాం హైదరాబాద్ విశ్వవిద్యాలయం తెలంగాణలో ఉన్నత విద్యకు ప్రసిద్ధి చెందిన కేంద్రంగా ఎదిగింది.  సాంకేతిక, వ్యాపార నిర్వహణ, ఔషధశాస్త్రం, వాస్తుశిల్పం మరియు ఇతర రంగాలలో విస్తృతమైన అకాడెమిక్ ప్రోగ్రామ్‌లను అందిస్తున్న ఏడు పాఠశాలలతో, గిటాం విద్యార్థులకు ఒక ఉత్తేజకరమైన మరియు సమగ్రమైన అభ్యాస అనుభవాన్ని అందిస్తుంది.  మా అధ్యాపకవర్గం అనుభవజ్ఞులైన నిపుణులతో నిండి ఉంది, వారు విద్యార్థుల సమగ్ర అభివృద్ధికి కట్టుబడి ఉన్నారు. మా ఆధునిక సదుపాయాలు, వినూత్న పద్ధతులు మరియు ప్రపంచ స్థాయి ప్రమాణాలు విద్యార్థులకు అత్యుత్తమ అకాడెమిక్ నాణ్యతను అందిస్తాయి.  గిటాం హైదరాబాద్, విజయవంతమైన మరియు సంతృప్తికరమైన ఉద్యోగాల కోసం తమ విద్యార్థులను సిద్ధం చేయడానికి కృషి చేస్తుంది.


##  Programs of Distinction

GITAM Hyderabad offers a wide array of undergraduate, postgraduate, and doctoral programs.  Some of our flagship programs include:

* **B.Com (ACCA):** Integrates commerce education with ACCA qualifications.
* **B.Pharm:** A four-year program focusing on pharmaceutical education and practical training.
* **B.Sc. Computer Science with Cognitive Systems:**  A cutting-edge program combining computer science with cognitive systems.
* **B.Sc. Food Science and Technology:**  Provides a comprehensive understanding of the food industry.
*  Many more programs across diverse disciplines including Engineering, Architecture,  Humanities & Social Sciences, Public Policy and Science.


##  Student Life

GITAM Hyderabad is more than just academics; it's a vibrant community that fosters personal growth and development.  Students enjoy a range of extracurricular activities, including sports, clubs, and events, creating a holistic learning experience.  Our state-of-the-art facilities, supportive faculty, and welcoming campus environment contribute to a thriving student culture.


## Careers

Our dedicated Career Guidance Centre helps students navigate their career paths through workshops, internships, and networking opportunities.  GITAM's strong industry connections ensure graduates are well-prepared for successful careers.

## Contact Us

Visit our website at [Insert Website Address Here] to learn more about GITAM Hyderabad and explore the opportunities that await you.


In [39]:
import time
from IPython.display import Markdown, display, clear_output

def create_brochure_typing(company_name, url):
    model = genai.GenerativeModel("gemini-1.5-flash")
    prompt = system_prompt + "\n\n" + get_brochure_user_prompt(company_name, url)
    response = model.generate_content(prompt)

    # Simulate typing output
    text = response.text
    display_handle = display(Markdown(""), display_id=True)
    result = ""
    for char in text:
        result += char
        time.sleep(0.005)  # adjust speed
        display_handle.update(Markdown(result))

In [40]:
create_brochure_typing('Gitam', 'https://www.gitam.edu/hyderabad/campus-life')

Found links: {'links': [{'type': 'about us', 'url': 'https://www.gitam.edu/about-us'}, {'type': 'careers page', 'url': 'https://careers.gitam.edu/'}, {'type': 'alumni page', 'url': 'https://www.alumni.gitam.edu/'}]}


# GITAM: Shaping Futures Through Education and Innovation

**A Brochure for Prospective Students, Investors, and Employees**

**(Insert GITAM Logo Here)**

##  About GITAM

GITAM (Deemed to be University) is a prestigious institution of higher education with campuses in Hyderabad, Bengaluru, and Visakhapatnam, along with the GITAM Institute of Medical Sciences and Research (GIMSR). Established in 2009 (Hyderabad campus), GITAM offers a wide array of undergraduate, postgraduate, and doctoral programs across diverse disciplines including:

* **Engineering:** Computer Science,  and more.
* **Management:** Business Administration, and more.
* **Science:** Chemistry, Physics, Computer Science with Cognitive Systems, Food Science and Technology, and more.
* **Humanities & Social Sciences:** Public Policy, and more.
* **Pharmacy:** B.Pharm.
* **Architecture:** Bachelor of Architecture.
* **Medicine, Nursing, Paramedical Science, Physiotherapy (GIMSR)**
* **Law (Visakhapatnam)**
* **Liberal Education and Performing Arts**

GITAM is committed to providing a vibrant learning environment fostering academic excellence, research, and innovation.  We boast esteemed faculty, state-of-the-art infrastructure, and prestigious partnerships that provide students with global exposure and opportunities.


##  Life at GITAM

**For Students:**

GITAM offers a holistic learning experience that extends beyond academics.  Our campuses are vibrant hubs of activity, hosting numerous intellectual, social, and cultural events throughout the year.  We encourage student engagement through chapters, clubs, sports, and recreational activities.  Our career guidance center (GCGC) provides support for career planning and placement assistance.  Top programs include B.Com (ACCA integrated), B.Pharm, and various B.Sc. options.


**For Faculty:**

GITAM values its faculty, providing opportunities for research, innovation, and professional development.  We are committed to creating a supportive and collaborative environment where faculty can excel in their teaching and research endeavors.  (Examples of faculty members and their specializations are available on the website.)

**For Investors:**

GITAM represents a strong investment opportunity in a rapidly growing sector. Our commitment to academic excellence, research and innovation, and our established presence across multiple locations makes GITAM a stable and promising venture.


**For Employees:**

GITAM offers a dynamic and rewarding work environment.  We are committed to diversity, equality, and inclusion, and we value the contributions of our employees. (Specific career opportunities are listed on our website's Careers section).


##  Our Commitment

GITAM is dedicated to sustainable development, incorporating principles of climate action, sustainable procurement, sustainable investment, and a net-zero commitment. We are also committed to equality, diversity, and inclusiveness through our various support cells.


## Contact Information

For further inquiries, please visit our website at [Insert Website Address Here] or contact us via:

* **General Inquiries:** [Insert General Contact Email or Phone Number Here]
* **Admissions:** [Insert Admissions Contact Email or Phone Number Here]
* **Career Opportunities:** [Insert Careers Contact Email or Phone Number Here]


**(Include QR code linking to the website)**


In [41]:
create_brochure_typing('Anttropic', 'https://www.anthropic.com/')

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}]}


# Anthropic: Building Safe and Beneficial AI

**A Brochure for Prospective Customers, Investors, and Recruits**

**Introduction:**

Anthropic is a leading AI safety and research company dedicated to building safe and beneficial artificial intelligence.  Our flagship product, Claude, is a powerful AI model available through various plans catering to individual users, teams, enterprises, and educational institutions.  We prioritize responsible AI development and strive to ensure our technologies serve humanity's long-term well-being.


**For Customers:**

* **Claude:**  Experience the power of Claude, our most intelligent AI model (Opus 4 and Sonnet 4), designed for collaboration, coding, and a wide range of tasks.  Choose from flexible plans to suit your needs: Max, Team, Enterprise, and Education.
* **API Access:** Integrate Claude directly into your applications to build custom AI-powered experiences. Comprehensive developer documentation and support are available.
* **Solutions:**  Discover how Claude can enhance your workflows across diverse areas, including AI agents, coding assistance, and customer support.
* **Customer Stories:** Learn how our customers are using Claude to achieve their objectives. (Link to customer stories page implied)

**For Investors:**

* **Cutting-Edge Research:** Anthropic is at the forefront of AI safety research, developing innovative techniques to ensure AI systems are aligned with human values.  Our research focuses on areas like interpretability, alignment science, and responsible scaling.
* **Strong Leadership:**  Our team comprises leading experts in AI, safety, and ethics.
* **Market Opportunity:** The demand for safe and reliable AI solutions is rapidly growing, and Anthropic is uniquely positioned to capitalize on this opportunity.
* **Responsible Development:** Our commitment to responsible AI development assures long-term sustainability and positive societal impact.

**For Recruits:**

* **Join a Mission-Driven Team:**  Be part of a team passionate about building the future of AI safety.
* **Challenging and Rewarding Work:** Contribute to impactful projects pushing the boundaries of AI technology.
* **Career Growth Opportunities:** Develop your skills and expertise in a dynamic and innovative environment.
* **Anthropic Academy:**  Expand your knowledge of AI development with our dedicated learning platform. (Link to Anthropic Academy implied)
* **Open Roles:** View our current job openings and apply today. (Link to careers page implied)


**Company Culture:**

Anthropic fosters a collaborative and inclusive environment where innovation and ethical considerations are paramount. We value intellectual curiosity, teamwork, and a commitment to our mission.

**Research Highlights:**

Our research focuses on:

* **AI Alignment:** Ensuring AI systems behave as intended and align with human values.
* **Interpretability:** Understanding how AI models reach their conclusions.
* **Responsible Scaling:** Developing policies and practices to ensure safe and beneficial AI scaling.
* **Economic Index:** Monitoring the economic impacts of AI.


**Contact Information:**

While not explicitly stated on provided text,  contact information can be found on the actual website through a “Speak with Sales” link on the landing page, and likely via a dedicated contact page.  Look for a general inquiries email or a sales contact form.


**Note:**  This brochure is based solely on the provided text from the Anthropic website.  A more comprehensive brochure would require accessing and reviewing additional information from the website.


In [ ]:
# question="Please explain this code if it is "

In [64]:
prom = "You are a personal assistant that explains the given code. If the code contains any errors, point them out clearly and provide a corrected version. If there are no errors, just explain what it does."

In [62]:
ques=input("Enter the code: ")
def generate_code(ques):
  model = genai.GenerativeModel("gemini-1.5-flash")
  prompt = prom + "\n\n" + ques
  response = model.generate_content(prompt)

  # Show the result
  display(Markdown(response.text))

Enter the code: public ListNode partition(ListNode head, int x) {         ListNode beforehead=new ListNode(0);         ListNode afterhead=new ListNode(0);          ListNode before=beforehead;         ListNode after=afterhead;


In [66]:
generate_code(ques)

This code snippet appears to be the beginning of a function that partitions a linked list.  It's designed to rearrange the nodes in a linked list such that all nodes with a value less than `x` come before nodes with a value greater than or equal to `x`.

Let's break down what each line does:

* `public ListNode partition(ListNode head, int x)`: This line declares a public method named `partition` that takes two arguments:
    * `head`:  A `ListNode` representing the head of the linked list.
    * `x`: An integer value used as the partition point.

* `ListNode beforehead = new ListNode(0);`: This creates a new `ListNode` called `beforehead` with a value of 0. This node will serve as a dummy head for the list of nodes with values less than `x`.  Using a dummy node simplifies the handling of edge cases (empty list).

* `ListNode afterhead = new ListNode(0);`: This creates a new `ListNode` called `afterhead` with a value of 0. This node will serve as a dummy head for the list of nodes with values greater than or equal to `x`.  Similar to `beforehead`, this simplifies the code.

* `ListNode before = beforehead;`: This sets the `before` pointer to point to `beforehead`.  `before` will be used to traverse and build the "less than x" list.

* `ListNode after = afterhead;`: This sets the `after` pointer to point to `afterhead`. `after` will be used to traverse and build the "greater than or equal to x" list.


**The code is incomplete.**  It's missing the crucial part that iterates through the input linked list, separates nodes based on the value `x`, and then connects the two sublists.  To make it complete, we need to add the iterative part.


Here's a corrected and complete version:

```java
public ListNode partition(ListNode head, int x) {
    ListNode beforehead = new ListNode(0);
    ListNode afterhead = new ListNode(0);
    ListNode before = beforehead;
    ListNode after = afterhead;

    while (head != null) {
        if (head.val < x) {
            before.next = head;
            before = before.next;
        } else {
            after.next = head;
            after = after.next;
        }
        head = head.next;
    }

    before.next = afterhead.next; // Connect the two sublists
    after.next = null; // Terminate the second list
    return beforehead.next; // Return the head of the partitioned list

}
```

This improved version iterates through the linked list, assigning nodes to either the `before` or `after` list based on their value relative to `x`. Finally, it connects the `before` and `after` lists and returns the head of the resulting partitioned list.  Remember that you'll also need a `ListNode` class definition for this code to compile and run.  A simple example would be:

```java
class ListNode {
    int val;
    ListNode next;
    ListNode(int x) { val = x; }
}
```
